In [1]:
from urllib import urlretrieve
from urllib2 import HTTPError, URLError, urlopen
import re
import urllib2
from bs4 import BeautifulSoup
from time import sleep
import sys

In [2]:
start_url_base = "https://www.gutenberg.org/wiki/Science_Fiction_by_Women_(Bookshelf)"
base_url = "https://www.gutenberg.org"

In [3]:
def make_soup(url):
    hdr = {"User-Agent": "Mozilla/5.0"}
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=hdr)).read(),'lxml')

In [4]:
def get_book_links(url):
    soup = make_soup(url)
    atagbooks = soup.find_all("a", attrs = {"title": re.compile(r'^ebook:')})
    book_links = []
    for element in atagbooks:
        book_link = "http:" + element["href"]
        book_links.append(book_link)
    return book_links        

In [ ]:
### main() for downloading book in .txt
## Change the path accordingly in the urlretrieve() function below

for booklink in get_book_links(start_url_base):
    print "Processing %s" %(booklink)
    try:
        soup = make_soup(booklink + '#download')
    except HTTPError:
        print "Time out."
        sleep(2.0)
        soup = make_soup(booklink + '#download')
    try:
        txtlink = "http:" + soup.find("a", {"type": re.compile(r"^text/plain")})['href']
    except TypeError:
        print "Failed to find the link for downloading %s" %(booklink)
        pass
    soup2 = make_soup(booklink + '#bibrec')
    foundth = soup2.find('th', string='Title')
    title = foundth.next_sibling.next_sibling.get_text().strip()
    try:
        urlretrieve(txtlink, 'txt_scifibookshelf11072017/' + title)
        print "%s downloaded." %(booklink)
    except:
        print "Failed to download %s" %(booklink)
        pass
    sleep(0.1)